<img src="https://i.imgur.com/gb6B4ig.png" width="400" alt="Weights & Biases" />

# Quickstart
Use Weights & Biases for machine learning experiment tracking, dataset versioning, and project collaboration.

<div><img /></div>

<img src="https://i.imgur.com/uEtWSEb.png" width="650" alt="Weights & Biases" />

<div><img /></div>

#Weights & Biases 💜 Ray Tune
Both Weights and Biases and Ray/Tune are built for scale and handle millions of models every month for teams doing some of the most cutting-edge deep learning research.

Whereas W&B is a centralized repository for everything you need to track, reproduce and gain insights from your models easily; Ray/Tune provides a simple interface for scaling and running distributed experiments. A few reasons why our community likes Ray/Tune –

* Simple Distributed execution: Ray Tune makes it easy to scale from a single node, to multiple GPUs, and further multiple nodes.
* Large number of algorithms: Ray Tune has a huge number of algorithms including Population Based Training, ASHA, and HyperBand
* Framework agnostic: Ray Tune works across frameworks including PyTorch, Keras, Tensorflow, XGBoost, and PyTorchLightning.
* Fault-tolerance: Ray Tune is built on top of Ray, providing fault tolerance out of the box.

Let's start by inastalling the libraries.


In [ ]:
!pip install -qq filelock==3.0.12
!pip install -U -qq ray==0.8.7
!pip install -qq wandb

# Reproducibility
In order to make this experiment reproducible, we'll set the seeds for random number generators of various libraries used in this experiment.

In [ ]:
import getpass
import random

import numpy as np
import torch

torch.backends.cudnn.deterministic = True
random.seed(hash("setting random seeds") % 2**32 - 1)
np.random.seed(hash("improves reproducibility") % 2**32 - 1)
torch.manual_seed(hash("by removing stochasticity") % 2**32 - 1)
torch.cuda.manual_seed_all(hash("so runs are repeatable") % 2**32 - 1)

# W&B Ray Tune Integration
**W&B** integrates with `Ray` by offering two lightweight standalone integrations:

1.   You can either use `WandbLogger`, which automatically logs metrics reported to Tune to the Wandb API along with the configurations of the experiment. Here is an example usage.
```python
 def train(config):
        ...
        tune.report(accuracy=acc) #This reported metric will be logged by WandbLogger to W&B dashboard
...
analysis = tune.run(train ,
                  loggers = [WandbLogger],
                  ...
                    )

2. You might run into scenarios where you'd like to log custom metrics, plots and outputs(including media files) directly to your W&B Dashboard. This can be accomplised by using `@wandb_mixin`.

 `@wandb_mixin` is a function decorator, which can be used with the function API. It automatically initializes the Wandb API with Tune’s training information. This allows you to use `wandb.log()` within the function scope that uses this decorator.
You can just use the Wandb API like you would normally do, e.g. using `wandb.log()` to log your training process, charts and media files.
Example usage:
```python
@wandb_mixin
def train(config):
      ...
      wandb.log({
                  "Accuracy" : acc,
                "Output" : wandb.Image(output_grid),
                })
...
tune.run(train,
          ...  
             )
```

**Both of these methods can be used together or independently.**





In [ ]:
from ray import tune
from ray.tune.examples.mnist_pytorch import ConvNet, get_data_loaders, test, train
from ray.tune.integration.wandb import wandb_mixin
import torch.optim as optim
import wandb

@wandb_mixin
def train_mnist(config):

    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    train_loader, test_loader = get_data_loaders()

    model = ConvNet()
    model.to(device)

    optimizer = optim.SGD(model.parameters(), lr=config["lr"], momentum=config["momentum"])
    
    for i in range(10):
        train(model, optimizer, train_loader, device=device)
        acc = test(model, test_loader, device=device)

        # When using WandbLogger, the metrics reported to tune are also logged in the W&B dashboard
        tune.report(mean_accuracy=acc)

        # @wandb_mixin enables logging custom metric using wandb.log()
        error_rate = 100*(1-acc)
        wandb.log({ "Error Rate " : error_rate})



Wandb configuration is done by passing a wandb key to the config parameter of `tune.run()` (see detailed example below).

The content of the `wandb` config entry is passed to `wandb.init()` as keyword arguments. The exception are the following settings, which are used to configure the `wandb` itself:
```
Parameters
api_key_file (str) – Path to file containing the Wandb API KEY.

api_key (str) – Wandb API Key. Alternative to setting api_key_file.
```

Wandb’s `group`, `run_id` and `run_name` are automatically selected by Tune, but can be overwritten by filling out the respective configuration values.

Please see here for all other valid configuration settings: https://docs.wandb.com/library/init

In [ ]:
from ray.tune.integration.wandb import WandbLogger

api_key = getpass.getpass("Enter your W&B apikey from https://wandb.ai/settings : ")

analysis = tune.run(
    train_mnist,
    loggers=[WandbLogger], # WandbLogger logs experiment configurations and metrics reported via tune.report() to W&B Dashboard 

     resources_per_trial={'gpu': 1},
     config={
        #wandb dict accepts all arguments that can be passed in wandb.init() 
        "wandb": {"project": "raytune", "job_type":'raytune-demo' , 'api_key': api_key},

        # Hyperparameters
        "lr": tune.grid_search([0.0001, 0.001, 0.1]),
        "momentum": tune.grid_search([0.9, 0.99])    
    })

print("Best config: ", analysis.get_best_config(metric="mean_accuracy"))

# Get a dataframe for analyzing trial results.
df = analysis.dataframe()